In [1]:
import os 
os.chdir('../..')
os.getcwd()

'/home/lanyanyan/xiongruibin/MatchZoo-py'

In [2]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.1


In [3]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss(num_neg=4,margin=0.1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [4]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [5]:
preprocessor = mz.models.HCRN.get_default_preprocessor()

In [6]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8571.95it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4260.02it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3152090.17it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6864.27it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4171.22it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 459781.13it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 6257.17it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [7]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fa33cb32290>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fa339d26e10>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [8]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [9]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [10]:
padding_callback = mz.models.HCRN.get_default_padding_callback()

device = torch.device('cuda:1')

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=100,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    num_workers=8,
    callback=padding_callback,
    device=device
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=160,
    stage='dev',
    sort=False,
    shuffle=False,
    num_workers=10,
    callback=padding_callback,
    device=device
)

In [11]:
model = mz.models.HCRN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix

model.params['hidden_size'] = 100
model.params['pooling_type'] = 'selective'
model.params['intra_attention'] = True
model.params['dropout_rate'] = 0.4

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

HCRN(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (word_proj): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): ReLU()
  )
  (lstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (intra_complex_proj): Sequential(
    (0): Linear(in_features=200, out_features=200, bias=True)
    (1): ReLU()
  )
  (complex_proj): Sequential(
    (0): Linear(in_features=400, out_features=400, bias=True)
    (1): ReLU()
  )
  (matching): Matching()
  (out): Linear(in_features=800, out_features=1, bias=True)
  (dropout): Dropout(p=0.4)
) 9410501


In [12]:
from pytorch_transformers import AdamW
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-8)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=5,
    data_parallel=False,
    save_all=True,
    save_dir='ckpt2',
    clip_norm=50.0,
    device=device
)

In [13]:
trainer.run()

[Iter-11 Loss-0.147]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4504 - normalized_discounted_cumulative_gain@5(0.0): 0.5149 - mean_average_precision(0.0): 0.4796



[Iter-22 Loss-0.110]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4385 - normalized_discounted_cumulative_gain@5(0.0): 0.5112 - mean_average_precision(0.0): 0.4731



[Iter-33 Loss-0.088]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5405 - normalized_discounted_cumulative_gain@5(0.0): 0.6039 - mean_average_precision(0.0): 0.5595



[Iter-44 Loss-0.051]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5481 - normalized_discounted_cumulative_gain@5(0.0): 0.5918 - mean_average_precision(0.0): 0.5412



[Iter-55 Loss-0.020]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5644 - normalized_discounted_cumulative_gain@5(0.0): 0.6081 - mean_average_precision(0.0): 0.5652

Cost time: 135.19425678253174s
